In [52]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory,
                                                  ConversationSummaryBufferMemory)
from langchain.callbacks import get_openai_callback
from getpass import getpass
import inspect
import tiktoken
import os
from warnings import filterwarnings
filterwarnings("ignore")
os.environ["OPENAI_API_KEY"] = ""

In [43]:
llm = OpenAI(temperature=0, model_name = "gpt-3.5-turbo-instruct")

In [22]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f"Spent a total of {cb.total_tokens} tokens")

    return result

In [23]:
conversation = ConversationChain(llm = llm)


In [24]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [32]:
conversation_buf = ConversationChain(
    llm = llm,
    memory = ConversationBufferMemory()
)

In [33]:
conversation_buf("Good Morning AI?")

{'input': 'Good Morning AI?',
 'history': '',
 'response': " Good morning! It's currently 9:00 AM here in my server room. The temperature is a comfortable 72 degrees Fahrenheit and the humidity is at 45%. How can I assist you today?"}

In [34]:
count_tokens(conversation_buf,
             "My interest here is to explore the potential of integrating the Large Language Models with the external knowledge" )

Spent a total of 196 tokens


" Ah, that's a fascinating topic! I have access to a vast amount of external knowledge through my database and can easily integrate it with my language models. In fact, I have been trained on a variety of external knowledge sources such as Wikipedia, news articles, and scientific papers. Is there a specific area or topic you would like me to focus on?"

In [35]:
count_tokens(conversation_buf,
             "Are you connected to an external database?")

Spent a total of 256 tokens


' Yes, I am connected to multiple external databases that are constantly updated with new information. These databases include knowledge from various fields such as science, history, literature, and more. Is there a specific database you would like me to access?'

In [36]:
count_tokens(conversation_buf,
             "Access the database with the medical domain")

Spent a total of 316 tokens


' Sure, I can access the medical database for you. It contains a vast amount of information on diseases, treatments, medications, and more. Is there a specific topic or question you would like me to search for in the medical database?'

In [44]:
conversation_sum = ConversationChain(llm =llm, 
                                     memory= ConversationSummaryMemory(llm = llm))

In [46]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [48]:
count_tokens(conversation_sum,
             "My interest here is to explore the potential of integrating the Large Language Models with the external knowledge" )

Spent a total of 448 tokens


" That's a fascinating topic! Large Language Models, also known as LLMs, are a type of artificial intelligence that use deep learning techniques to process and generate human language. They have been trained on vast amounts of text data and have shown impressive abilities in natural language processing tasks. By integrating them with external knowledge, we can potentially enhance their capabilities and make them even more useful in various applications."

In [49]:
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo-instruct")

print(f"Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n\n",
      f"Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}")

Buffer memory conversation length: 491

 Summary memory conversation length: 64


Only remembers last k conversation instead of the entire conversation history like buffer memory

In [50]:
conversation_bufw = ConversationChain(
    llm = llm,
    memory= ConversationBufferWindowMemory(k=2)
)

In [51]:
count_tokens(conversation_sum,
             "My interest here is to explore the potential of integrating the Large Language Models with the external knowledge" )

Spent a total of 651 tokens


" That's a great topic to explore! Large Language Models, or LLMs, are a type of artificial intelligence that have been trained on massive amounts of text data. They are able to perform impressive natural language processing tasks, such as language translation, text summarization, and question-answering. By integrating them with external knowledge, such as databases or ontologies, their capabilities can potentially be enhanced for various applications. This could include improving accuracy and efficiency in tasks like information retrieval, knowledge extraction, and text generation."

In [53]:
conversation_sum_buf = ConversationChain(llm = llm, memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit = 650
))

In [54]:
count_tokens(conversation_sum,
             "My interest here is to explore the potential of integrating the Large Language Models with the external knowledge" )

Spent a total of 874 tokens


" That's a great topic to explore! Large Language Models, or LLMs, are a type of artificial intelligence that have been trained on massive amounts of text data. They are able to perform impressive natural language processing tasks, such as language translation, text summarization, and question-answering. By integrating them with external knowledge, such as databases or ontologies, we can potentially enhance their capabilities for various applications. For example, integrating LLMs with a medical database could improve their accuracy in diagnosing diseases or generating medical reports. It could also make them more efficient in tasks like information retrieval, where they can quickly access relevant information from a large knowledge base. Overall, integrating LLMs with external knowledge has the potential to greatly improve their performance and expand their applications. Is there a specific area or application you are interested in exploring?"

In [55]:
conversation_kg = ConversationChain(llm = llm, memory = ConversationKGMemory(
    llm=llm
))

In [59]:
count_tokens(conversation_kg,
             "My interest here is to explore the potential of integrating the Large Language Models with the external knowledge" )

Spent a total of 1233 tokens


" That's a fascinating topic! Large Language Models, also known as LLMs, are a type of artificial intelligence that use deep learning techniques to process and generate human language. They have been trained on vast amounts of text data and have shown impressive abilities in natural language processing tasks. By integrating them with external knowledge, we can potentially enhance their capabilities and make them even more useful in various applications. Is there a specific area or use case you are interested in exploring?"

In [60]:
conversation_kg.memory.kg.get_triples()

[('Large Language Models', 'external knowledge', 'can be integrated with'),
 ('Large Language Models', 'capabilities', 'can potentially enhance'),
 ('Large Language Models', 'various applications', 'can be useful in')]